In [0]:
#importa bibliotecas
from pyspark.sql.functions import col, to_date

# Lê os dados brutos da camada bronze
df_bronze = spark.read.json('/Volumes/raw/ipca_bronze/ipca_raw')

# Cria o schema (caso ainda não exista)
spark.sql("CREATE SCHEMA IF NOT EXISTS silver")

# Dropar a tabela caso já exista com schema antigo
spark.sql("DROP TABLE IF EXISTS silver.ipca_silver")

# Tratamento: conversão de tipos
df_silver = (df_bronze
    .withColumn("data", to_date(col("data"), "dd/MM/yyyy"))
    .withColumn("valor", col("valor").cast("double"))  
    .dropDuplicates()        
    .na.drop()                                        
)

# Salva o dataframe tratado como tabela delta na camada silver
(df_silver.distinct()  # remove duplicatas
   .coalesce(1) # grava em um único arquivo (opcional)
   .write
   .format('delta')
   .mode('overwrite')
   .saveAsTable('silver.ipca_silver')  # salva no catálogo dentro do schema silver
)